In [4]:
from selenium import webdriver  
import time 
import re
from lxml import etree
from bs4 import BeautifulSoup
import bs4 as bs
from selenium.common.exceptions import NoSuchElementException 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
import pandas as pd

URLs_brand = []
#browser = webdriver.Firefox()#Chrome('./chromedriver.exe') 
whole = ['2-a','3-b','4-c','5-d','6-e','7-f','8-g','9-h','10-i','11-j','12-k','13-l','14-m','15-n','16-o','17-p',
        '18-q','19-r','20-s','21-t','22-u','23-v','24-w','25-x','26-y','27-z'] 

# For every page in the interval
for wh in whole:
    PATIENCE_TIME = 60 
    driver = webdriver.Chrome() 
    driver.get('https://www.nosetime.com/pinpai/'+str(wh)+'.html') 
    driver.encoding = 'utf-8'
    driver.maximize_window()

    html = driver.page_source
    soup = BeautifulSoup(html,'lxml')
    url_containers = soup.find_all('div', class_='odorlist')

    for container in url_containers:
        #find all the date of the news
        try: 
            for url_b in container.find_all('a'): 
                url_brand = url_b.get('href')
                URLs_brand.append(url_brand)
        except: 
            url_brand = "no_url"
            URLs_brand.append(url_brand)
    driver.close()
driver.quit() 

In [5]:
print(URLs_brand[:10])

['/pinpai/10091761-oulong-atelier-cologne.html', '/pinpai/10091761-oulong-atelier-cologne.html', '/pinpai/10056110-annikeguteer-annick-goutal.html', '/pinpai/10056110-annikeguteer-annick-goutal.html', '/pinpai/10051198-paermazhishui-acqua-di-parma.html', '/pinpai/10051198-paermazhishui-acqua-di-parma.html', '/pinpai/10020871-aimu-amouage.html', '/pinpai/10020871-aimu-amouage.html', '/pinpai/10035498-annasu-anna-sui.html', '/pinpai/10035498-annasu-anna-sui.html']


In [9]:
len(URLs_brand)

4080

In [71]:
Cleaned_URls_brand = set(URLs_brand)
len(Cleaned_URls_brand)

2040

In [39]:
#get the pure id of brand
Brand_id = []
for i in Cleaned_URls_brand:
    brand_id = re.sub(r'[^0-9]','',i)
    Brand_id.append(brand_id)
datapage1 = {
    'Brand_id': Brand_id,
}
brandid = pd.DataFrame.from_dict(datapage1, orient ='index').transpose()
print(brandid.info())
brandid.to_csv('Brand_id.csv',index = False, encoding = 'utf_8_sig')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2040 entries, 0 to 2039
Data columns (total 1 columns):
Brand_id    2040 non-null object
dtypes: object(1)
memory usage: 16.0+ KB
None


In [74]:
import csv
Brand_id=[]
with open('Brand_id.csv', 'r') as f:
    reader = csv.reader(f)
    for line in f:
        Brand_id = [row[0] for row in reader]
        print(Brand_id[:10])

['10014016', '10078016', '10054734', '10093177', '10075765', '10038916', '10015637', '10068357', '10063657', '10010461']


# 这是有多少个品牌

In [83]:
len(Brand_id)

2040

In [84]:
set(Brand_id)
len(Brand_id)

2040

In [75]:
#获得page >1的所有urls
URLs = []
#browser = webdriver.Firefox()#Chrome('./chromedriver.exe') 
import numpy as np
import random
import time
from time import sleep
# For every page in the interval
for bid in Brand_id:
    PATIENCE_TIME = 60 
    driver = webdriver.Chrome()
    driver.get('https://www.nosetime.com/brand.php?id='+str(bid))
    driver.encoding = 'utf-8'
    driver.maximize_window()
    
    html = driver.page_source
    soup = BeautifulSoup(html,'lxml')
    all_url_container = soup.find_all('div', class_='next_news')
    
    for container in all_url_container:
        try: 
            for url in container.find_all('a'): 
                URLs.append("https://www.nosetime.com/brand.php?id="+str(bid)+"page="+url.text)
                
        except: 
            urls = "no_allurls"
            URLs.append(allurls)
        driver.close()
        time.sleep(1)
driver.quit() 

In [130]:
len(URLs) 

3424

In [135]:
#删除URL中不需要的，比如含有下一页和尾页的URL
p_urls = []
error = []
for i in AllURLs:
    if not "下一页" in i:
        if not "尾页" in i:
            p_urls.append(i)
    else: error.append(i)
len(p_urls)  

2226

In [136]:
#检查这些不需要的URLs
print(error[:10])

['https://www.nosetime.com/brand.php?id=10014016page=下一页', 'https://www.nosetime.com/brand.php?id=10054734page=下一页', 'https://www.nosetime.com/brand.php?id=10010461page=下一页', 'https://www.nosetime.com/brand.php?id=10061680page=下一页', 'https://www.nosetime.com/brand.php?id=10029483page=下一页', 'https://www.nosetime.com/brand.php?id=10098532page=下一页', 'https://www.nosetime.com/brand.php?id=10035498page=下一页', 'https://www.nosetime.com/brand.php?id=10030919page=下一页', 'https://www.nosetime.com/brand.php?id=10075007page=下一页', 'https://www.nosetime.com/brand.php?id=10077602page=下一页']


In [137]:
#将香水的page>1的url存入一个csv
datapage2 = {
    'p_urls': p_urls,
}
p_urls = pd.DataFrame.from_dict(datapage2, orient ='index').transpose()
print(p_urls.info())
p_urls.to_csv('part_urls.csv',index = False, encoding = 'utf_8_sig')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2226 entries, 0 to 2225
Data columns (total 1 columns):
p_urls    2226 non-null object
dtypes: object(1)
memory usage: 17.5+ KB
None


In [142]:
#导入该csv
import csv
purls=[]
with open('part_urls.csv', 'r') as f:
    reader = csv.reader(f)
    for line in f:
        purls = [row[0] for row in reader]

In [153]:
import re
part_id = []
for i in purls:
    newi = re.split('\d+$',i) #移除url最后的数字
    newi1 = re.sub(r'[^0-9]','',newi[0])
    part_id.append(newi1)

In [157]:
part_id = set(part_id)
len(part_id)

599

In [161]:
#这里是香水page=1的所有url。
another_url = []
for i in Brand_id: #all_brand_id
    if i not in part_id: #find the singel page brand id
        another_url.append("https://www.nosetime.com/brand.php?id="+str(i)) #append the single page url
len(another_url) #the number is correct

1441

# 这是有多少个URL

In [163]:
whole_urls = purls + another_url
len(whole_urls)#2226+1441=3667

3667

In [164]:
#write all the urls into the csv files
datapage3 = {
    'whole_urls ': whole_urls,
}
whole_urls = pd.DataFrame.from_dict(datapage3, orient ='index').transpose()
print(whole_urls.info())
whole_urls.to_csv('whole_urls.csv',index = False, encoding = 'utf_8_sig')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3667 entries, 0 to 3666
Data columns (total 1 columns):
whole_urls     3667 non-null object
dtypes: object(1)
memory usage: 28.7+ KB
None
